In [1]:
from nltk import text
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os
import cv2

In [2]:
def datalist():
    mylist = os.listdir("ISL_Dataset_C/")
    ls = []
    for x in mylist:
        ls.append(os.path.splitext(x)[0])
    return ls

In [3]:
def cartoonize(img, ds_factor=4, sketch_mode=False):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.medianBlur(img_gray, 7)
    edges = cv2.Laplacian(img_gray, cv2.CV_8U, ksize=5)
    ret, mask = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY_INV)
    if sketch_mode:
        return cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    img_small = cv2.resize(img, None, fx=1.0/ds_factor, fy=1.0/ds_factor, interpolation=cv2.INTER_AREA)
    num_repetitions = 10
    sigma_color = 5
    sigma_space = 7
    size = 5

    for i in range(num_repetitions):
        img_small = cv2.bilateralFilter(img_small, size, sigma_color, sigma_space)

    img_output = cv2.resize(img_small, None, fx=ds_factor, fy=ds_factor, interpolation=cv2.INTER_LINEAR)
    dst = np.zeros(img_gray.shape)

    dst = cv2.bitwise_and(img_output, img_output, mask=mask)
    return dst

In [4]:
def convert(text):
    text = text.title()
    #print(text)
    word_Lemmatized = WordNetLemmatizer()
    text = word_Lemmatized.lemmatize(text)
    #print(text)
    ls = datalist()
    #print(ls)
    if(text == 'Bye'):
        print('Good Bye !!')
        exit
    elif(text in ls):
        cap = cv2.VideoCapture('ISL_Dataset_C/' + text + '.mp4')
        if (cap.isOpened()== False): 
            print("Error opening video file")
            exit
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret == True:
                cv2.imshow('Corresponding Video', cartoonize(frame, sketch_mode=False))
                if cv2.waitKey(50) & 0xFF == ord('q'):
                    break
            else: 
                break
        cap.release()
        cv2.destroyAllWindows()
    else:
        print("Sorry, currently we don't have the word in our back end !")

In [7]:
print('Enter text: ')
text = input()
convert(text)

Enter text: 
England
